In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

## Modelling
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.model_selection import train_test_split, KFold, cross_val_score, learning_curve
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from catboost import CatBoostRegressor, Pool
from sklearn.svm import SVR
import xgboost as xgb


In [2]:
frequency_domain_features_train = pd.read_csv('C:/Users/mypc/Documents/Learning/Great Learning/Hackathon/Train Data/frequency_domain_features_train.csv')

In [3]:
heart_rate_non_linear_features_train = pd.read_csv('C:/Users/mypc/Documents/Learning/Great Learning/Hackathon/Train Data/heart_rate_non_linear_features_train.csv')

In [4]:
time_domain_features_train = pd.read_csv('C:/Users/mypc/Documents/Learning/Great Learning/Hackathon/Train Data/time_domain_features_train.csv')

In [5]:
print(frequency_domain_features_train.shape)
print(heart_rate_non_linear_features_train.shape)
print(time_domain_features_train.shape)

(369289, 12)
(369289, 7)
(369289, 20)


In [6]:
data_train_temp = pd.merge(frequency_domain_features_train,heart_rate_non_linear_features_train, on = 'uuid')
print(data_train_temp.shape)

(369289, 18)


In [7]:
data_train = pd.merge(data_train_temp,time_domain_features_train, on = 'uuid')
print(data_train.shape)

(369289, 37)


In [8]:
data_train.head()

,uuid,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci,datasetId,condition,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR
0,89df2855-56eb-4706-a23b-b39363dd605a,2661.894136,72.203287,1009.249419,27.375666,98.485263,15.522603,0.421047,1.514737,3686.666157,65.018055,0.015380,11.001565,199.061782,2.139754,1.163485,2,no stress,885.157845,853.763730,140.972741,15.554505,15.553371,9.063146,69.499952,11.133333,0.533333,-0.856554,0.335218,-0.000203,-0.000179,0.017080,0.007969,0.007969,2.143342,-0.856554,0.335218
1,80c795e4-aa56-4cc0-939c-19634b89cbb2,2314.265450,76.975728,690.113275,22.954139,99.695397,2.108525,0.070133,0.304603,3006.487251,327.296635,0.003055,9.170129,114.634458,2.174499,1.084711,2,interruption,939.425371,948.357865,81.317742,12.964439,12.964195,6.272369,64.363150,5.600000,0.000000,-0.408190,-0.155286,-0.000059,0.000611,0.013978,0.004769,0.004769,2.930855,-0.408190,-0.155286
2,c2d5d102-967c-487d-88f2-8b005a449f3e,1373.887112,51.152225,1298.222619,48.335104,98.950472,13.769729,0.512671,1.049528,2685.879461,94.280910,0.010607,11.533417,118.939253,2.135350,1.176315,2,interruption,898.186047,907.006860,84.497236,16.305279,16.305274,5.182201,67.450066,13.066667,0.200000,0.351789,-0.656813,-0.000011,-0.000263,0.018539,0.008716,0.008716,2.127053,0.351789,-0.656813
3,37eabc44-1349-4040-8896-0d113ad4811f,2410.357408,70.180308,1005.981659,29.290305,98.224706,18.181913,0.529387,1.775294,3434.520980,55.328701,0.018074,11.119476,127.318597,2.178341,1.179688,2,no stress,881.757865,893.460030,90.370537,15.720468,15.720068,5.748591,68.809562,11.800000,0.133333,-0.504947,-0.386138,0.000112,0.000494,0.017761,0.008660,0.008660,2.050988,-0.504947,-0.386138
4,aa777a6a-7aa3-4f6e-aced-70f8691dd2b7,1151.177330,43.918366,1421.782051,54.242160,96.720007,48.215822,1.839473,3.279993,2621.175204,29.487873,0.033912,13.590641,87.718281,2.221121,1.249612,2,no stress,809.625331,811.184865,62.766242,19.213819,19.213657,3.266724,74.565728,20.200000,0.200000,-0.548408,-0.154252,-0.000100,-0.002736,0.023715,0.013055,0.013055,1.816544,-0.548408,-0.154252


In [9]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 369289 entries, 0 to 369288
Data columns (total 37 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   uuid               369289 non-null  object 
 1   VLF                369289 non-null  float64
 2   VLF_PCT            369289 non-null  float64
 3   LF                 369289 non-null  float64
 4   LF_PCT             369289 non-null  float64
 5   LF_NU              369289 non-null  float64
 6   HF                 369289 non-null  float64
 7   HF_PCT             369289 non-null  float64
 8   HF_NU              369289 non-null  float64
 9   TP                 369289 non-null  float64
 10  LF_HF              369289 non-null  float64
 11  HF_LF              369289 non-null  float64
 12  SD1                369289 non-null  float64
 13  SD2                369289 non-null  float64
 14  sampen             369289 non-null  float64
 15  higuci             369289 non-null  float64
 16  da

In [10]:
df_train = data_train.drop(['uuid'],axis = 1)

In [11]:
df_train.describe()

,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci,datasetId,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR
count,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.0,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,3.692890e+05,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000,369289.000000
mean,2199.580170,64.289242,946.530252,34.095182,95.566718,39.245603,1.615576,4.433282,3185.356025,115.977200,0.048506,10.593708,154.178997,2.062471,1.182292,2.0,846.650104,841.965890,109.352531,14.977498,14.976767,7.396597,73.941824,9.841143,0.866001,0.523235,0.041628,-1.756587e-06,-0.000465,0.018571,0.009701,0.009701,2.006817,0.523235,0.041628
std,1815.773422,16.774844,574.171780,16.040290,4.123365,45.398869,1.761073,4.123365,1923.227187,360.855129,0.049238,2.914795,109.170222,0.206999,0.062192,0.0,124.603984,132.321005,77.117025,4.120766,4.120768,5.143834,10.337453,8.195574,0.990189,1.790348,0.699522,1.630256e-04,0.000868,0.005455,0.003897,0.003897,0.375845,1.790348,0.699522
min,159.480176,19.031219,90.048557,2.165119,69.879083,0.061783,0.002150,0.012825,377.692795,2.319952,0.000128,3.911344,38.307745,0.434576,1.033984,2.0,547.492221,517.293295,27.233947,5.529742,5.529630,2.660381,48.737243,0.000000,0.000000,-1.894820,-2.136278,-1.233914e-03,-0.004425,0.008987,0.003220,0.003220,1.169342,-1.894820,-2.136278
25%,1001.189280,52.909877,545.449386,22.305936,93.645734,10.720312,0.346803,1.228054,1828.147788,14.737458,0.012433,8.368340,90.326864,2.032977,1.139929,2.0,760.228533,755.750735,64.205641,11.830959,11.830671,4.541896,66.715776,3.666667,0.000000,-0.352783,-0.359291,-7.281695e-05,-0.000917,0.014261,0.006984,0.006984,1.749801,-0.352783,-0.359291
50%,1667.903111,66.350237,782.716291,32.047025,96.643140,24.841938,1.039513,3.356860,2796.856587,28.789747,0.034735,10.196621,116.221063,2.134214,1.174293,2.0,822.951438,819.689595,82.608243,14.415918,14.415388,5.952112,74.217809,7.600000,0.466667,0.040736,-0.060966,-9.330777e-07,-0.000312,0.017318,0.008691,0.008691,1.934416,0.040736,-0.060966
75%,2654.121052,76.825032,1201.432256,44.647115,98.771946,45.272368,2.245115,6.354266,4052.260157,80.429614,0.067854,12.679005,166.764850,2.181929,1.223621,2.0,924.117422,916.821570,118.237002,17.927144,17.924839,7.919841,80.334937,13.333333,1.466667,0.722833,0.282417,6.911667e-05,0.000131,0.021827,0.011460,0.011460,2.221232,0.722833,0.282417
max,12617.977191,97.738848,3291.548112,77.928847,99.987175,364.486936,13.095664,30.120917,13390.684098,7796.443096,0.431043,18.836107,796.852945,2.234841,1.361219,2.0,1322.016957,1653.122250,563.486949,26.629477,26.629392,54.523950,113.752309,39.400000,5.466667,64.088107,6.777800,1.244098e-03,0.002095,0.036571,0.026955,0.026955,3.724134,64.088107,6.777800


In [12]:
def missing_check(df):
    total = df.isnull().sum().sort_values(ascending=False)   # total number of null values
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)  # percentage of values that are null
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])  # putting the above two together
    return missing_data # return the dataframe
missing_check(df_train)

,Total,Percent
SKEW_REL_RR,0,0.0
KURT_REL_RR,0,0.0
datasetId,0,0.0
higuci,0,0.0
sampen,0,0.0
SD2,0,0.0
SD1,0,0.0
HF_LF,0,0.0
LF_HF,0,0.0
TP,0,0.0


In [13]:
## User defined function to calculate statistics for the attributes
def stats_measure(x):
  print('Number of Unique Values')
  print(x.nunique())
  print(" ")
  print('Summary of the attribute')
  print(x.describe())
  print(" ")
  print('Skewness of the attribute')
  if x.dtype=='O':
    print('Categorical Variable')
  else:
    print(x.skew())
  print(" ")
  print('Kurtosis of the attribute')
  if x.dtype=='O':
    print('Categorical Variable')
  else:
    print(x.kurt())   


In [14]:
for p in df_train.columns:
  print('-'*100)  
  print(f'Statistical Measures of the attribute: ',p)
  col = df_train[p]
  stats_measure(col)
  print('-'*100)


----------------------------------------------------------------------------------------------------
Statistical Measures of the attribute:  VLF
Number of Unique Values
369289
 
Summary of the attribute
count    369289.000000
mean       2199.580170
std        1815.773422
min         159.480176
25%        1001.189280
50%        1667.903111
75%        2654.121052
max       12617.977191
Name: VLF, dtype: float64
 
Skewness of the attribute
1.9607349536265608
 
Kurtosis of the attribute
4.493563049525889
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Statistical Measures of the attribute:  VLF_PCT
Number of Unique Values
369289
 
Summary of the attribute
count    369289.000000
mean         64.289242
std          16.774844
min          19.031219
25%          52.909877
50%          66.350237
75%          76.825032
max          97.738848
Na

count        369289
unique            3
top       no stress
freq         200082
Name: condition, dtype: object
 
Skewness of the attribute
Categorical Variable
 
Kurtosis of the attribute
Categorical Variable
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Statistical Measures of the attribute:  MEAN_RR
Number of Unique Values
369289
 
Summary of the attribute
count    369289.000000
mean        846.650104
std         124.603984
min         547.492221
25%         760.228533
50%         822.951438
75%         924.117422
max        1322.016957
Name: MEAN_RR, dtype: float64
 
Skewness of the attribute
0.6479996931060378
 
Kurtosis of the attribute
0.4343821626576201
----------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------

1.2587841008510972
 
Kurtosis of the attribute
1.434218030422703
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Statistical Measures of the attribute:  SDSD_REL_RR
Number of Unique Values
369289
 
Summary of the attribute
count    369289.000000
mean          0.009701
std           0.003897
min           0.003220
25%           0.006984
50%           0.008691
75%           0.011460
max           0.026955
Name: SDSD_REL_RR, dtype: float64
 
Skewness of the attribute
1.2587844630384302
 
Kurtosis of the attribute
1.4342211739455308
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Statistical Measures of the attribute:  SDRR_RMSSD_REL_RR
Number of Unique Values
369289
 
Summary of the att

In [15]:
# List the numerical and categorical columns
numeric_cols = df_train.select_dtypes(include = [np.number]).columns.tolist()
categorical_cols = df_train.select_dtypes(include = [np.object]).columns.tolist()
print('The numeric attributes are:', numeric_cols)
print('The categorical attributes are:', categorical_cols)

The numeric attributes are: ['VLF', 'VLF_PCT', 'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT', 'HF_NU', 'TP', 'LF_HF', 'HF_LF', 'SD1', 'SD2', 'sampen', 'higuci', 'datasetId', 'MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD', 'SDRR_RMSSD', 'HR', 'pNN25', 'pNN50', 'KURT', 'SKEW', 'MEAN_REL_RR', 'MEDIAN_REL_RR', 'SDRR_REL_RR', 'RMSSD_REL_RR', 'SDSD_REL_RR', 'SDRR_RMSSD_REL_RR', 'KURT_REL_RR', 'SKEW_REL_RR']
The categorical attributes are: ['condition']


In [ ]:
df_train['condition'].value_counts()

In [25]:
dummy_var = pd.get_dummies(df_train['condition'],drop_first = True)
dummy_var.head()

,no stress,time pressure
0,1,0
1,0,0
2,0,0
3,1,0
4,1,0


In [26]:
df_train = pd.concat([df_train, dummy_var], axis = 1)
df_train = df_train.drop('condition', axis = 1)

In [27]:
df_train.head()

,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci,datasetId,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,no stress,time pressure
0,2661.894136,72.203287,1009.249419,27.375666,98.485263,15.522603,0.421047,1.514737,3686.666157,65.018055,0.015380,11.001565,199.061782,2.139754,1.163485,2,885.157845,853.763730,140.972741,15.554505,15.553371,9.063146,69.499952,11.133333,0.533333,-0.856554,0.335218,-0.000203,-0.000179,0.017080,0.007969,0.007969,2.143342,-0.856554,0.335218,1,0
1,2314.265450,76.975728,690.113275,22.954139,99.695397,2.108525,0.070133,0.304603,3006.487251,327.296635,0.003055,9.170129,114.634458,2.174499,1.084711,2,939.425371,948.357865,81.317742,12.964439,12.964195,6.272369,64.363150,5.600000,0.000000,-0.408190,-0.155286,-0.000059,0.000611,0.013978,0.004769,0.004769,2.930855,-0.408190,-0.155286,0,0
2,1373.887112,51.152225,1298.222619,48.335104,98.950472,13.769729,0.512671,1.049528,2685.879461,94.280910,0.010607,11.533417,118.939253,2.135350,1.176315,2,898.186047,907.006860,84.497236,16.305279,16.305274,5.182201,67.450066,13.066667,0.200000,0.351789,-0.656813,-0.000011,-0.000263,0.018539,0.008716,0.008716,2.127053,0.351789,-0.656813,0,0
3,2410.357408,70.180308,1005.981659,29.290305,98.224706,18.181913,0.529387,1.775294,3434.520980,55.328701,0.018074,11.119476,127.318597,2.178341,1.179688,2,881.757865,893.460030,90.370537,15.720468,15.720068,5.748591,68.809562,11.800000,0.133333,-0.504947,-0.386138,0.000112,0.000494,0.017761,0.008660,0.008660,2.050988,-0.504947,-0.386138,1,0
4,1151.177330,43.918366,1421.782051,54.242160,96.720007,48.215822,1.839473,3.279993,2621.175204,29.487873,0.033912,13.590641,87.718281,2.221121,1.249612,2,809.625331,811.184865,62.766242,19.213819,19.213657,3.266724,74.565728,20.200000,0.200000,-0.548408,-0.154252,-0.000100,-0.002736,0.023715,0.013055,0.013055,1.816544,-0.548408,-0.154252,1,0


In [28]:
df_train_imputed = df_train.copy(deep = True)

In [29]:
for col in numeric_cols:
    Q3 = df_train_imputed[col].quantile(0.75)
    Q1 = df_train_imputed[col].quantile(0.25)
    IQR = Q3 - Q1
    upper_lim = Q3 + (3 * IQR)
    lower_lim = Q1 - (3 * IQR)
    df_train_imputed.loc[(df_train_imputed[col] < lower_lim) | (df_train_imputed[col] > upper_lim), col] = df_train_imputed[col].median()

#df_train_imputed.isnull().sum()

In [30]:
# Using SimpleImputer to fill missing values by median
print('Descriptive Stats before handling outliers: \n', '--'*30)
display(df_train.describe().T)

print('Descriptive Stats after handling outliers: \n', '--'*30)
display(df_train_imputed.describe().T)


Descriptive Stats before handling outliers: 
 ------------------------------------------------------------


,count,mean,std,min,25%,50%,75%,max
VLF,369289.0,2199.580170,1815.773422,159.480176,1001.189280,1.667903e+03,2654.121052,12617.977191
VLF_PCT,369289.0,64.289242,16.774844,19.031219,52.909877,6.635024e+01,76.825032,97.738848
LF,369289.0,946.530252,574.171780,90.048557,545.449386,7.827163e+02,1201.432256,3291.548112
LF_PCT,369289.0,34.095182,16.040290,2.165119,22.305936,3.204703e+01,44.647115,77.928847
LF_NU,369289.0,95.566718,4.123365,69.879083,93.645734,9.664314e+01,98.771946,99.987175
HF,369289.0,39.245603,45.398869,0.061783,10.720312,2.484194e+01,45.272368,364.486936
HF_PCT,369289.0,1.615576,1.761073,0.002150,0.346803,1.039513e+00,2.245115,13.095664
HF_NU,369289.0,4.433282,4.123365,0.012825,1.228054,3.356860e+00,6.354266,30.120917
TP,369289.0,3185.356025,1923.227187,377.692795,1828.147788,2.796857e+03,4052.260157,13390.684098
LF_HF,369289.0,115.977200,360.855129,2.319952,14.737458,2.878975e+01,80.429614,7796.443096


Descriptive Stats after handling outliers: 
 ------------------------------------------------------------


,count,mean,std,min,25%,50%,75%,max
VLF,369289.0,2005.718669,1426.877298,159.480176,1001.189280,1.667903e+03,2496.617475,7612.783844
VLF_PCT,369289.0,64.289242,16.774844,19.031219,52.909877,6.635024e+01,76.825032,97.738848
LF,369289.0,944.821855,571.037542,90.048557,545.449386,7.827163e+02,1199.536481,3169.378593
LF_PCT,369289.0,34.095182,16.040290,2.165119,22.305936,3.204703e+01,44.647115,77.928847
LF_NU,369289.0,95.698710,3.805403,78.269762,93.729698,9.664314e+01,98.771946,99.987175
HF,369289.0,33.564026,32.819622,0.061783,10.720312,2.484194e+01,41.551803,148.921411
HF_PCT,369289.0,1.508332,1.508504,0.002150,0.346803,1.039513e+00,2.137243,7.938278
HF_NU,369289.0,4.301290,3.805403,0.012825,1.228054,3.356860e+00,6.270302,21.730238
TP,369289.0,3147.309366,1836.098125,377.692795,1828.147788,2.796857e+03,4024.752713,10724.569836
LF_HF,369289.0,47.087608,53.973996,2.319952,14.737458,2.878975e+01,52.132280,277.505613


In [31]:
# Splitting the dataset into the Training and Test sets 
from sklearn.model_selection import train_test_split
X = df_train.drop(['HR'],axis = 1)
y = df_train['HR']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((258502, 36), (110787, 36), (258502,), (110787,))

In [32]:
## Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
X_train = pd.DataFrame(X_train, columns = X.columns)
X_test = pd.DataFrame(X_test, columns = X.columns)

In [33]:
X_train.head()

,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci,datasetId,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,no stress,time pressure
0,-0.337147,0.959736,-1.054647,-1.048462,-1.815251,0.152738,0.408434,1.815251,-0.629175,-0.300201,1.763130,-1.414366,-0.072301,-2.174314,0.608114,0.0,-1.564410,-1.625997,-0.075356,-1.414529,-1.414366,0.760703,-0.900278,-0.873594,2.122184,2.903013,-0.312152,0.672531,-0.766268,-0.070457,-0.070459,-1.276527,2.122184,2.903013,0.919629,-0.458159
1,-0.592396,-1.296683,0.902949,1.328997,0.209151,0.331519,0.247530,-0.209151,-0.282537,-0.246284,-0.232994,1.043280,-0.606898,0.764165,1.122707,0.0,-0.307765,-0.241200,-0.602181,1.043199,1.043280,-0.803039,1.281231,-0.671444,-0.597993,-0.281476,-1.070699,-2.609775,0.964367,0.872310,0.872309,-0.498063,-0.597993,-0.281476,0.919629,-0.458159
2,-0.368810,1.059505,-1.139790,-1.148010,-1.980925,0.064074,0.364668,1.980925,-0.686538,-0.301451,1.943323,-1.504964,-0.080670,-3.482454,0.407653,0.0,-1.656869,-1.718888,-0.083830,-1.505027,-1.504964,0.839507,-0.932838,-0.873594,2.787550,3.393158,-0.834047,0.360072,-0.848841,-0.173727,-0.173726,-1.228314,2.787550,3.393158,0.919629,-0.458159
3,2.223109,1.296993,0.101254,-1.261970,0.906393,-0.709788,-0.862387,-0.906393,2.113033,0.071285,-0.842984,0.460188,0.651853,-0.240933,-1.617371,0.0,0.807937,0.672997,0.651109,0.460012,0.460188,0.401060,0.100937,1.147909,0.329040,-0.395905,-0.109019,-0.034041,0.009207,-0.709902,-0.709898,1.801348,0.329040,-0.395905,-1.087395,2.182649
4,-0.230664,-0.644641,0.714834,0.412462,-1.961633,3.418022,2.389290,1.961633,0.075501,-0.301313,1.922196,0.423610,0.098634,-0.680353,0.333699,0.0,-1.648950,-1.258674,0.098316,0.423460,0.423610,-0.078152,-0.102562,1.484826,-0.483919,-1.140503,-0.706962,0.734183,1.813500,2.250219,2.250219,-1.237276,-0.483919,-1.140503,0.919629,-0.458159
